<div class="header" id="Sartorius">Sartorius - Cell Instance Segmentation<a class="anchor-link" href="#Sartorius"></a></div>

<img src="https://storage.googleapis.com/kaggle-competitions/kaggle/30201/logos/header.png?t=2021-09-03-15-27-46.png"
     alt="Markdown neurone picture"
     style="" />

<div class="smallheader" id="Inhoud">Danny Rovers<a class="anchor-link" href="#Inhoud"></a></div>
<br></br>
<div class="smallheader" id="Inhoud">Thomas Hendriks<a class="anchor-link" href="#Inhoud"></a></div>
<br></br>
<div class="smallheader" id="Inhoud">Liam Freke<a class="anchor-link" href="#Inhoud"></a></div>

In [1]:
%%html
<style>.header {padding: 20px; text-align: center;background: #d564d9;
color: white; font-size: 30px; font-weight: bold;}
.smallheader {color: #d564d9;
font-size: 25px;} </style>

<div class="header" id="Inhoud">Inhoud<a class="anchor-link" href="#Inhoud"></a></div>

1. [Opdracht context](#Context)
1. [Neuronen](#Neuronen)
1. [Alzheimer](#Alzheimer)
1. [Hersen tumor](#Hersentumor)
1. [Libraries](#Libraries)
1. [Data verkenning](#Imports)
1. [Model](#Imports)
1. [Uitvoering](#Imports)
1. [Conclusie](#Imports)
1. [Discussie](#Imports)
0. [Referenties](#Referenties)

<div class="header" id="Context">Opdracht context<a class="anchor-link" href="#Context"></a></div>

<div class='smallheader'>Vanuit school<div>

Van de hogeschool hebben we als opdracht gekregen om door middel van imageprocessing annotaties te maken uit foto’s van neuronen. Artsen maken nu handmatig een annotatie met behulp van een foto. Het toepassen van computer algoritme om dit proces te automatiseren zou de artsen veel tijd besparen. Als een algoritme een hoge accuracy kan waarmaken is het mogelijk dit proces sneller en goedkoper te laten verlopen.

De dataset komt uit een Kaggle competitie die eerder is afgesloten. We gebruiken deze dataset alsnog om onze kennis over imageprocessing te vergroten.

<div class='smallheader'>Competitie overzicht<div>

De Kaggle competitie heeft als doel om in plaatjes relevante objecten te herkennen en uit te lichten. Deze plaatjes zijn van neuronen cell types die vaak gebruikt worden bij onderzoek naar neurologische aandoeningen. In de competitie is het doel om een model te trainen die met behulp van training images segmentatie van neuronen cellen kan uitvoeren. Hoe hoger de accuraatheid hoe beter.

Als dit succesvol is wordt het onderzoek naar neurobiologie versterkt. Onderzoekers zullen gemakkelijker interessente objecten kunnen waarnemen in neuronen waardoor de kans dat ziektes eerder worden waargenomen toeneemt. Dit maakt het mogelijk om sneller deze ziektes te behandelen en misschien zelfs sneller nieuwe medicijnen te ontwikkelen.

>**Link naar de Kaggle competitie**
>- [Sartorius - Cell Instance Segmentation](https://www.kaggle.com/competitions/sartorius-cell-instance-segmentation/overview/description) 

<div class='smallheader'>Evaluatie<div>

<mark>Nog vertalen en in normaal Nederlands uitleggen (vragen voor de les)</mark>

<img src="https://929687.smushcdn.com/2633864/wp-content/uploads/2016/09/iou_examples.png?lossy=1&strip=1&webp=1"
     alt="Markdown Alzheimer picture"
     style=""/>

<mark>In de eerste formule wordt uitgerekend wat de overlap is tussen de voorspellende waarde en de echte waarde. Bij AnB staat de waarde van de mask die ons model voorspelt. Bij AuB staat de exacte waarde van de mask die berekent is door de dokter. Hieruit komt een waarde onder de 1. De waardes tussen 0,5 en 0,95 worden gezien als accuraat.</mark>

<mark>Alle waardes die tussen 0,5 en 0,95 worden gezien als hit en zijn dus true positives (TP). De waardes buiten 0,5 tot 0,95 worden gezien als false negatives(FN) of false positives (FP). De false negatives zijn waardes die ons model voorspelt heeft maar niet kloppen. En de false positives zijn waardes die ons model niet voorspelt heeft maar wel had moeten voorspellen.</mark>

This competition is evaluated on the mean average precision at different intersection over union (IoU) thresholds. The IoU of a proposed set of object pixels and a set of true object pixels is calculated as:

<div class="math">
\begin{equation}
  IoU(A,B) = \frac{A \cap B}{ A \cup B}.
\end{equation}
</div>

The metric sweeps over a range of IoU thresholds, at each point calculating an average precision value. The threshold values range from 0.5 to 0.95 with a step size of ```0.05: (0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95).``` In other words, at a threshold of 0.5, a predicted object is considered a "hit" if its intersection over union with a ground truth object is greater than 0.5.

At each threshold value ***t***, a precision value is calculated based on the number of true positives (TP), false negatives (FN), and false positives (FP) resulting from comparing the predicted object to all ground truth objects:

<div class="math">
\begin{equation}
  \frac{TP(t)}{TP(t) + FP(t) + FN(t)}.
\end{equation}
</div>

A true positive is counted when a single predicted object matches a ground truth object with an IoU above the threshold. A false positive indicates a predicted object had no associated ground truth object. A false negative indicates a ground truth object had no associated predicted object. The average precision of a single image is then calculated as the mean of the above precision values at each IoU threshold:

<div class="math">
\begin{equation}
  \frac{1}{|thresholds|} \sum_t \frac{TP(t)}{TP(t) + FP(t) + FN(t)}.
\end{equation}
</div>

Lastly, the score returned by the competition metric is the mean taken over the individual average precisions of each image in the test dataset.

>**Extra informatie / bronnen betreft de formules**
>- [Intersection over Union (IoU) for object detection](https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/) 

<div class="header" id="Neuronen">Neuronen<a class="anchor-link" href="#Neuronen"></a></div>

In de dataset worden er drie verschillende cell types gedifineerd. Het is belangrijk om de context te weten van deze cellen en waar ze voor dienen.

<img style="text-align: left" src="https://storage.googleapis.com/kaggle-media/competitions/Sartorius/Sartorius_Competition%20Description%20Image%20350x379.png"
     alt="Markdown Alzheimer picture"
     style="" />

<div class='smallheader'>SH-SY5Y<div>

De cell type ```SH-SY5Y (Neuroglioblastoma)``` is een cell lijn die afgestamd is van mensen. De originele cell lijn genaamd ```SK-N-SH``` is gekloond vanuit de beenmerg van een 4 jaar oud kind met neuroblastoma. ```SH-SY5Y``` cellen worden vaak gebruikt om ziektes zoals Parkinson's, neurogenesis en andere ziektes te bestuderen.

<div class='smallheader'>Astrocytes<div>

De cell type ```Astrocytes (Astroglia)``` is ster gevormde *glial cell* die te vinden is in het brein en in de ruggenmerg. 

Ze hebben diverse functies zoals: 
- Voedingstoffen vervoeren naar zenuwweefsel.
- Onderhouden van het Ion balans.
- Regulatie van de bloedsomloop in het brein.
- Onderdeel van de reparatie van schade in het brein of ruggenmerg.

<div class='smallheader'>Neuronen van Cerebral Cortex<div>

De ```Cerebral Cortex (Cerebral Mantle)``` is de buitenste laag van de grote hersenen binnen het brein. De ```Cerebral Cortex``` bevat tussen de 14 en 16 biljoen *Neuronen*, deze *Neuronen* onderzoeken wij.

Het heeft diverse functies zoals:

- Connecties tussen brein delen.
- Voorziet Sensorische functies.
- Voorziet Motorsche functies.

*Neuronen* binnen de ```Cerebral Cortex``` kunnen onderzocht worden, hiermee kunnen verschillende ziektes herkent worden zoals: Alzheimer's, Lafora, Nerurologische ziektes, Epilepsy en nog veel meer.

>**Extra informatie / bronnen betreft de cel types**
>- [SH-SY5Y](https://en.wikipedia.org/wiki/SH-SY5Y) 
>- [Astrocyte](https://en.wikipedia.org/wiki/Astrocyte)
>- [Cerebral cortex](https://en.wikipedia.org/wiki/Cerebral_cortex#Function) 

<div class="header" id="Alzheimer">Alzheimer ziekte<a class="anchor-link" href="#Alzheimer"></a></div>

<img src="https://herseninstituut.nl/wp-content/uploads/2018/06/Alzheimer-pixabay.jpg"
     alt="Markdown Alzheimer picture"
     style="" />

<div class='smallheader'>Wat is Alzheimer?<div>

Alzheimer is de meest voorkomende oorzaak van dementie. Alzheimer is de oorzaak van 60-80% van de
dementie gevallen (Alzheimer's assosiation, n.d.).

De grootste bekende risicofactor is toenemende leeftijd. De meerderheid van de mensen die alzheimer
hebben is 65 jaar of ouder.

Alzheimer wordt erger over de tijd heen. In de beginfase van de ziekte is het geheugenverlies minimaal,
maar gedurende de tijd zal het geheugenverlies toenemen. Ook zal een persoon niet meer kunnen
communiceren en reageren op zijn omgeving.

Alzheimer heeft nog geen geneesmiddel. De eerste stappen binnen het maken van een werkend
geneesmiddel voor de beginfase van de ziekte zijn gemaakt. Er is namelijk een behandeling
(Aducanumab) die werkt bij het verwijderen van amyloid, wat een van de kenmerken van de ziekte is
(Alzheimer's assosiation, n.d.). 

<div class='smallheader'>Hoe kun je Alzheimer herkennen?<div>

Microscopische veranderingen in de hersenen beginnen lang voor de eerste tekenen van
geheugenverlies.

De hersenen hebben 100 miljard zenuwcellen (neuronen). Elke neuron verbind met veel andere
neuronen om communicatie netwerken te vormen. Dit maakt alles wat een mens doet mogelijk.
Alzheimer is te herkennen door middel van (Alzheimer Nederland, n.d.):
- Vergeetachtigheid;
- Problemen bij het dagelijks handelen;
- Taalproblemen;
- Kwijtraken van spullen;
- Slecht beoordelingsvermogen;
- Terugtrekken uit sociale activiteiten;
- Onrust.

Artsen kunnen door naar microscopische foto’s te kijken de eerste tekenen van het ontwikkelen van
Alzheimer herkennen.

<div class='smallheader'>Hoe werken neuronen precies?<div>

Neuronen communiceren met elkaar door elektrische signalen in hun axons te sturenen. Axons zijn de lange aanhangsels die aan het midden van de neuron vastzit. Als het elektrische signaal bij het einde van de axon komt geeft het een chemisch signaal af die wordt opgevangen door de axon van de tegenoverliggende cel.

<div class='smallheader'>Hoe is Alzheimer te herkennen op cel niveau?<div>

De cellen van alzheimer patiënten worden langzaam gevuld met het proteïne Tau, wat ervoor zorgt dat er ophopingen ontstaan in de neuronen. Verder wordt het proteïne beta-amyloid buiten de neuron opgehoopt in plaques. Deze plaques kunnen de communicatie tussen neuronen blokkeren. Deze combinatie samen zorgt waarschijnlijk voor een giftig reactie in de hersenen. Waardoor axonen van neuronen geen signalen kunnen overbrengen en dus langzaam afsterven. In afbeeldingen kan je dit zien aan verminderde axonen van de neuron.

<div class='smallheader'>De oorzaak van alzheimer.<div>

Wetenschappers geloven dat alzheimer voorkomt dat gedeeltes van de fabriek van een neuron goed
blijven werken. Er is echter nog niet duidelijk hoe dit proces in gang wordt gezet. Wel heeft leeftijd
invloed op de kans dat je Alzheimer ontwikkeld (Alzheimer's assosiation, n.d.).

>**Extra informatie / bronnen betreft Alzheimer**
>- [Alzheimer Nederland. (sd). Herkennen en symptomen van dementie](https://www.alzheimer-nederland.nl/dementie/herkennen-symptomen) 
>- [Alzheimer's assosiation. (sd). What is Alzheimer’s Disease?](https://www.alz.org/alzheimers-dementia/what-is-alzheimers)
>- [What Happens to the Brain in Alzheimer's Disease?](https://www.nia.nih.gov/health/what-happens-brain-alzheimers-disease) 
>- [Pyramidal cell axon initial segment in Alzheimer´s disease](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9130508/) 

<div class="header" id="Hersentumor">Hersen tumor <a class="anchor-link" href="#Hersentumor"></a></div>

<div class='smallheader'>Wat is een Hersentumor?<div>

Een tumor (gezwel) in de hersenen ontstaat wanneer hersencellen ongecontroleerd gaan delen en
groeien (Kanker.nl, n.d.). Een hersentumor kan goedaardig of kwaadaardig zijn. Bij kwaadaardige
tumoren praat je over een kanker. Een kwaadaardige hersentumor heet ook wel hersenkanker.

Een kwaadaardige hersentumor kan uitzaaien naar andere delen van de hersens of het ruggenmerg. Een
kwaadaardige hersentumor zaait zelden uit naar andere delen van het lichaam (Kanker.nl, n.d.).

<div class='smallheader'>De symptomen van een hersentumor.<div>

Een tumor in de hersenen kan invloed hebben op de werking van de hersenen en daardoor klachten
geven. Dat geldt zowel voor kwaadaardige als goedaardige tumoren in de hersenen.

Veelvoorkomende symptomen van een hersentumor zijn:
- Aanhoudende hoofdpijn;
- Verandering in verstandelijk vermogen en gedrag;
- Uitvalsverschijnselen;
- Epileptische aanvallen.

De symptomen van een hersentumor verschillen heel erg per persoon. De grootte, groeisnelheid en de
plaats hebben sterk invloed op de symptomen die iemand ervaart (Kanker.nl, n.d.).

<div class='smallheader'>Hoe is een hersen tumor te herkennen op cel niveau?<div>

Er zijn verschillende types van hersen tumoren die in de hersenen ontstaan. In de hersenen komen type 1 en 2 het meeste voor. Type 3 komt heel soms en voor en type 4 komt niet uit zichzelf voor. Type 1 is een langzaam groeiende tumor die erg moeilijk te herkennen is omdat ze erg lijken op gewonnen cellen. Type 1 cellen zijn langzaam groeiend en kan lang mee geleefd worden. Type 2 is een iets sneller groeiende cel die er iets anders uit ziet dan normale cellen en dus nog moeilijk te herkennen is, maar beter herkenbaar is dan type 1 cellen. Type 3 cellen groeien hard en hebben een erg ander uiterlijk dan normale cellen.

De vormen die ontstaan in neuronen zijn bijna altijd van type 1 en 2 in bevinden zich in de hippocampus. De vervormingen die te zien zijn bij neuronen zijn vervormingen van de cellen en andere kleur.

<div class='smallheader'>Wat zijn de oorzaken van een hersentumor?<div>

De oorzaak van hersentumoren is nog steeds onbekend. Er is wel gedacht dat een virus, straling, een
ongelukken of het contact met chemische stoffen de oorzaken kunnen zijn. Dit is echter nog niet
bewezen. Er is soms wel een erfelijke component, met meerdere patiënten met gezwellen in één familie
(Maxima mc, n.d.).

>**Extra informatie / bronnen betreft hersen tumors**
>- [Wat is een hersentumor?](https://www.kanker.nl/kankersoorten/hersentumoren/algemeen/wat-is-een-hersentumor#:~:text=Een%20tumor%20(gezwel)%20in%20de,dan%20bij%20veel%20andere%2) 
>- [Hersentumor](https://www.mmc.nl/oncologie/aandoeningen-en-behandelingen/hersentumor/#:~:text=De%20oorzaak%20van%20echte%20of,met%20gezwellen%20in%20%C3%A9%C3%A9n%20familie.)

<div class="header" id="Libraries">Libraries<a class="anchor-link" href="#Libraries"></a></div>

<div class="header" id="Referenties">Referenties<a class="anchor-link" href="#Referenties"></a></div>

> **Context:**
>- [Wat is een hersentumor?](https://www.kanker.nl/kankersoorten/hersentumoren/algemeen/wat-is-een-hersentumor#:~:text=Een%20tumor%20(gezwel)%20in%20de,dan%20bij%20veel%20andere%2) 
>- [Hersentumor](https://www.mmc.nl/oncologie/aandoeningen-en-behandelingen/hersentumor/#:~:text=De%20oorzaak%20van%20echte%20of,met%20gezwellen%20in%20%C3%A9%C3%A9n%20familie.)
>- [Alzheimer Nederland. (sd). Herkennen en symptomen van dementie](https://www.alzheimer-nederland.nl/dementie/herkennen-symptomen) 
>- [Alzheimer's assosiation. (sd). What is Alzheimer’s Disease?](https://www.alz.org/alzheimers-dementia/what-is-alzheimers)
>- [What Happens to the Brain in Alzheimer's Disease?](https://www.nia.nih.gov/health/what-happens-brain-alzheimers-disease) 
>- [Pyramidal cell axon initial segment in Alzheimer´s disease](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC9130508/)
>- [SH-SY5Y](https://en.wikipedia.org/wiki/SH-SY5Y) 
>- [Astrocyte](https://en.wikipedia.org/wiki/Astrocyte)
>- [Cerebral cortex](https://en.wikipedia.org/wiki/Cerebral_cortex#Function) 
>- [Intersection over Union (IoU) for object detection](https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/)  
>
> **Code:**
>- [🧬Sartorius:🧬 InDepth EDA + Explanation + Model](https://www.kaggle.com/code/ishandutta/sartorius-indepth-eda-explanation-model/notebook) 
>- [Understanding Run Length Encoding and Decoding](https://www.kaggle.com/code/susnato/understanding-run-length-encoding-and-decoding/notebook)
>- [Everything about Mask R-CNN: A Beginner's Guide](https://viso.ai/deep-learning/mask-r-cnn/) 
>- [🦠 Sartorius - Starter Baseline Torch U-net [>0.0]](https://www.kaggle.com/code/julian3833/sartorius-starter-baseline-torch-u-net-0-0?scriptVersionId=78653910)